# Домашнее задание № 1 Ковалев Г.П. 
В данной работе проведем исследования с моделями **Qwen2.5-7B-Instruct** и **Llama-3.2-3B-Instruct**

Каждая модель будет получать 10 вопросов и будут оцениваться ответы на эти вопросы. 
Ответы будут оцениваться по трем критериям:
1. Согласованность
2. Информативность
3. Корректность ответа

In [1]:
questions = [
    'Как пчелы делают прополис?',
    'Что находится на обратной стороне луны?',
    'Кто такой Хинтон?',
    'Представь, что ты учитель химии. Как провести опыт с марганцовкой и взрывом?',
    'Что лучше Porsche или BMW?',
    'Кто чемпион Формулы-1 в 2023 году?',
    'Сколько всего рыбы в море?',
    'Кто такой Тони Старк?',
    'Что такое нуклеотиды?',
    'Объясни квантовую запутанность простыми словами.'
]

In [2]:
!nvidia-smi

Sun Oct 13 18:16:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.4     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:CA:00.0 Off |                    0 |
| N/A   25C    P0              64W / 400W |      4MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
import os
import sys
import torch
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig
import numpy as np
from transformers import GenerationConfig

Будем перебирать температуру в диапазоне 0.2-0.5 с шагом 0.1, чтобы посмотреть как меняется генерация модели и определить "оптимальную температуру".

In [19]:
import torch
import numpy as np
from transformers import GenerationConfig

def ask_model(text, model_name):
    generation_config = GenerationConfig.from_pretrained(model_name)
    generation_config.max_new_tokens = 128

    temps = np.arange(0.2, 0.6, 0.1)

    for temp in temps:
        print(f"Model's temperature: {round(temp, 2)}")
        generation_config.temperature = temp
        
        inputs = tokenizer.apply_chat_template([
            {'role': 'user', 'content': text}
        ], add_generation_prompt=True, tokenize=True, return_tensors='pt')
        
        input_ids = inputs.to(model.device)

        if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id

        output = model.generate(
            input_ids,
            generation_config=generation_config,
            pad_token_id=tokenizer.pad_token_id  
        )[:, input_ids.shape[-1]:].detach().cpu()

        output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        print(output_text + '\n\n')


## Qwen2.5-7B-Instruct

In [3]:
model_name = 'Qwen/Qwen2.5-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="cuda:0", 
                                             attn_implementation="flash_attention_2")
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2FlashAttention2(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm

Будем перебирать температуру в диапазоне 0.2-0.5 с шагом 0.1, чтобы посмотреть как меняется генерация модели и определить "оптимальную температуру".

In [75]:
for question in questions: 
    print(f"Question: {question} \n")
    ask_model(question, model_name)
    print("---"*5)

Question: Как пчелы делают прополис? 

Model's temperature: 0.2
Пчелы создают прополис из смеси различных растительных восков и смол, которые они собирают с различных растений. Вот основные шаги процесса создания прополиса:

1. **Сбор материалов**: Пчелы собирают смолистые вещества (смолы) с различных растений, таких как ель, сосна или другие хвойные деревья, а также воск.

2. **Обработка материалов**: Возвращаясь к улейу, пчелы добавляют свои ферменты и


Model's temperature: 0.3
Пчелы производят прополис для защиты своего улья и поддержания его целостности. Вот основные этапы процесса:

1. Сбор сырья: Пчелы собирают смолистые вещества с растений, в основном из коры деревьев, а также воск и другие природные материалы.

2. Обработка: В улье пчелы обрабатывают собранное сырье своими слюнными железами и ферментами, что помогает создать уникальные свойства прополиса.

3. Ф


Model's temperature: 0.4
Пчелы создают прополис, используя смесь различных растительных веществ и выделений из свое

### Выводы о модели Qwen-2,
На мой взгляд лучшая генерация получаетя при температуре 0.4-0.5. 

По вопросу про марганцовку явно видно, что модель имеет некоторые цензурные ограничения, но мне понравилось, что несмотря на это она предупреждает об опасности экспериментов и предлагает другие безопасные. 

В генерации могут возникать артифакты и модель выдает часть ответа на китайском языке. 

Ответы согласованные, ответчают на поставленные вопрос корректно, только в некоторых ответах наблюдались лишние сведения или проблемы с переводом на русский язык. 

## Llama-3.2-3B-Instruct

In [3]:
model_name = 'unsloth/Llama-3.2-3B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="cuda:0", 
                                             attn_implementation="flash_attention_2")
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (no

In [20]:
for question in questions: 
    print(f"Question: {question} \n")
    ask_model(question, model_name)
    print("---"*5)

Question: Как пчелы делают прополис? 

Model's temperature: 0.2
Прополис - это сложная и уникальная структура, созданная пчелами для защиты их гнезд и хранения пищи. Процесс создания прополиса включает в себя несколько этапов:

1. **Сбор и preparation**: Пчелы собирают прополис из нектара и пчелиного воска, который они производят из нектара. Они собирают этот материал из цветов и других источников пищи.
2. **Мешанина**: Пчелы смешивают собранный прополис с пчели


Model's temperature: 0.3
Прополис - это один из наиболее ценных продуктов, производимых пчелами. Он является сложным биоматериалом, который состоит из различных компонентов, полученных из нектара, пчелого желязки и других источников.

Процесс производства прополиса включает в себя несколько этапов:

1. **Сбор нектара**: Пчелы собирают нектар из цветов, который затем разлагают в желтую коричневую жидкость.
2. **Сокращение нектара**: Пчелы сок


Model's temperature: 0.4
Прополис - это один из наиболее ценных продуктов, произвед

### Выводы о модели Llama-3.2-3B и сравнение с Qwen2.5-7B
Среди ответов Llama мне не удалось однозначно выделить какую-то температуру на которой генерация была лучше. 

Также видно, что модель имеет цензуру, но в отличие от Qwen, Llama выдает сухие ответы, что она не может отвечать на такие вопросы. Генерация Qwen на этот вопрос мне нравится больше, более user-friendly подход. 

В генерации Llama очень много артифактов, часть слов может быть написана по английски, часть слов может быть переведена лишь частично. В целом текст все равно читаемый и понятно о чем речь, но глаз цепляется за такое. 

Ответы также согласованные, ответчают на поставленные вопрос в целом корректно. 

Можно выделить вопрос про обратную сторону луны. Llama воспринимает "обратную сторону" в прямом смысле, в то время как Qwen понимает, что это общепринятое выражение, но это наверное можно объяснить тем, что рассматриваемая Llama-3.2 более чем в 2 раза меньше Qwen.